#### Dataset Creation

In [1]:
import random
import torch
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [2]:
from data_loading.models_dataset import EcoreModelDataset
from data_loading.models_dataset import ArchiMateModelDataset

config_params = dict(
    timeout = 120,
    min_edges = 10
)
ecore_model_dataset = EcoreModelDataset('ecore_555', reload=False, **config_params)
archimate_model_dataset = ArchiMateModelDataset('eamodelset', reload=True, **config_params)
# dataset = EcoreModelDataset('modelset', reload=False, remove_duplicates=True, **config_params)
# dataset = EcoreModelDataset('mar-ecore-github', reload=True, **config_params)

Loading ecore_555 from pickle
Loaded ecore_555 with 282 graphs
Loaded ecore_555 with 282 graphs
Graphs: 282


Loading Eamodelset:   0%|          | 0/979 [00:00<?, ?it/s]

Saving eamodelset to pickle
Saved eamodelset to pickle
Loaded eamodelset with 936 graphs
Graphs: 936


In [3]:
from data_loading.graph_dataset import GraphEdgeDataset, GraphNodeDataset

ecore_graph_edge_dataset = GraphEdgeDataset(
    ecore_model_dataset,
    use_attributes=True,
    use_edge_types=True,
    reload=True,
)

ecore_graph_node_dataset = GraphNodeDataset(
    ecore_model_dataset,
    use_attributes=True,
    reload=True,
)

Creating graphs:   0%|          | 0/282 [00:00<?, ?it/s]

Processing graphs:   0%|          | 0/282 [00:00<?, ?it/s]

Train edge classes: {0: 19479, 2: 3193, 1: 3696}
Test edge classes: {0: 4812, 2: 760, 1: 881}


Creating graphs:   0%|          | 0/282 [00:00<?, ?it/s]

Processing graphs:   0%|          | 0/282 [00:00<?, ?it/s]

Node label: abstract
Train Node classes: {1: 2212, 0: 7127}
Test Node classes: {0: 1890, 1: 594}


In [4]:
from data_loading.graph_dataset import GraphEdgeDataset, GraphNodeDataset

ecore_graph_edge_dataset = GraphEdgeDataset(
    archimate_model_dataset,
    use_edge_types=True,
    reload=True,
)

ecore_graph_node_dataset = GraphNodeDataset(
    archimate_model_dataset,
    use_attributes=True,
    reload=True,
)

Creating graphs:   0%|          | 0/936 [00:00<?, ?it/s]

Processing graphs:   0%|          | 0/936 [00:00<?, ?it/s]

Train edge classes: {7: 9421, 2: 13034, 10: 10833, 9: 7781, 0: 9578, 3: 8365, 4: 15936, 8: 21158, 6: 2383, 1: 6210, 5: 2341}
Test edge classes: {9: 1958, 3: 1972, 4: 3873, 8: 5192, 10: 2689, 0: 2386, 7: 2360, 2: 3187, 6: 583, 1: 1536, 5: 546}


Creating graphs:   0%|          | 0/936 [00:00<?, ?it/s]

Processing graphs:   0%|          | 0/936 [00:00<?, ?it/s]

Node label: type
Train Node classes: {51: 1816, 58: 1223, 41: 4102, 40: 3022, 2: 2689, 56: 3530, 43: 3040, 32: 7002, 10: 5288, 44: 1541, 6: 7069, 27: 834, 31: 432, 63: 2943, 16: 2256, 30: 252, 24: 854, 1: 414, 3: 2009, 57: 168, 17: 368, 47: 4016, 46: 374, 36: 3480, 28: 307, 13: 1735, 22: 1369, 45: 171, 52: 179, 53: 207, 23: 925, 8: 674, 14: 415, 29: 1156, 11: 501, 37: 597, 50: 995, 25: 1141, 20: 490, 4: 1550, 38: 957, 18: 2156, 9: 365, 62: 180, 42: 871, 33: 370, 39: 2754, 60: 252, 12: 258, 54: 768, 48: 110, 15: 126, 5: 50, 49: 145, 59: 33, 26: 250, 19: 63, 21: 206, 0: 323, 34: 49, 35: 67, 7: 79, 55: 90, 61: 127}
Test Node classes: {19: 17, 2: 699, 6: 1965, 32: 1760, 56: 1014, 63: 736, 44: 377, 51: 484, 58: 322, 41: 1001, 10: 1401, 31: 128, 40: 830, 47: 931, 13: 430, 36: 866, 45: 38, 39: 728, 23: 207, 8: 171, 27: 210, 17: 98, 37: 131, 25: 253, 11: 113, 0: 90, 22: 313, 3: 465, 43: 833, 50: 238, 18: 533, 38: 229, 28: 80, 4: 406, 20: 126, 14: 108, 33: 110, 29: 293, 16: 614, 42: 213, 9: 85,

In [1]:
import pandas as pd

df = pd.read_csv('datasets/sap_sam_2022/models/UML2.2Class.csv')

In [5]:
models = df['Model JSON'].unique().tolist()

In [6]:
import json


model = json.loads(models[0])

In [7]:
print(json.dumps(model, indent=4))

{
    "resourceId": "canvas",
    "properties": {
        "orientation": "horizontal",
        "description": "",
        "name": ""
    },
    "stencil": {
        "id": "Diagram"
    },
    "childShapes": [
        {
            "resourceId": "sid-D42AEB9B-E010-4AD9-9721-E99BBC29EC64",
            "properties": {
                "color": "#ffffff",
                "description": "",
                "classname": "layout\n\n",
                "keyword": "",
                "attributes": "",
                "attributescomplex": {
                    "totalCount": 1,
                    "items": [
                        {
                            "multiplicity": "",
                            "visibility": "protected",
                            "name": "matrix",
                            "property": "",
                            "defaultvalue": "",
                            "type": "String[][]"
                        }
                    ]
                },
              